In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
from pathlib import Path
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [18]:
# ==================== Dataset ====================
class GoPRODataset(Dataset):
    def __init__(self, blur_dir, sharp_dir, transform=None):
        self.blur_dir = Path(blur_dir)
        self.sharp_dir = Path(sharp_dir)
        self.transform = transform

        # Get all blur images
        blur_images = list(self.blur_dir.glob('*.png')) + list(self.blur_dir.glob('*.jpg'))

        # Get all sharp images
        sharp_images = list(self.sharp_dir.glob('*.png')) + list(self.sharp_dir.glob('*.jpg'))

        print(f"Found {len(blur_images)} blur images")
        print(f"Found {len(sharp_images)} sharp images")

        # Create a mapping by filename
        blur_dict = {img.name: img for img in blur_images}
        sharp_dict = {img.name: img for img in sharp_images}

        # Find common filenames
        common_names = sorted(set(blur_dict.keys()) & set(sharp_dict.keys()))

        if len(common_names) == 0:
            raise ValueError("No matching image pairs found! Check that blur and sharp folders have same filenames.")

        if len(common_names) < len(blur_images) or len(common_names) < len(sharp_images):
            print(f"Warning: Only {len(common_names)} matching pairs found")
            print(f"Missing in sharp: {len(blur_images) - len(common_names)}")
            print(f"Missing in blur: {len(sharp_images) - len(common_names)}")

        # Create matched pairs
        self.blur_images = [blur_dict[name] for name in common_names]
        self.sharp_images = [sharp_dict[name] for name in common_names]

        print(f"Using {len(self.blur_images)} matched image pairs")

    def __len__(self):
        return len(self.blur_images)

    def __getitem__(self, idx):
        blur_img = Image.open(self.blur_images[idx]).convert('RGB')
        sharp_img = Image.open(self.sharp_images[idx]).convert('RGB')

        if self.transform:
            blur_img = self.transform(blur_img)
            sharp_img = self.transform(sharp_img)

        return blur_img, sharp_img


In [19]:
# ==================== FPN Backbone ====================
class FPNBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, 1)

    def forward(self, x):
        return self.conv(x)

class FPNHead(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels, out_channels, 3, padding=1)
        )

    def forward(self, x):
        return self.block(x)

In [20]:
# ==================== DeblurGAN-v2 Generator ====================
class ResidualBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.InstanceNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(channels, channels, 3, padding=1),
            nn.InstanceNorm2d(channels)
        )

    def forward(self, x):
        return x + self.block(x)

class DeblurGenerator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, num_residual=9):
        super().__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.ReflectionPad2d(3),
            nn.Conv2d(in_channels, 64, 7),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),

            nn.Conv2d(64, 128, 3, stride=2, padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),

            nn.Conv2d(128, 256, 3, stride=2, padding=1),
            nn.InstanceNorm2d(256),
            nn.ReLU(inplace=True)
        )

        # Residual blocks
        self.residual_blocks = nn.Sequential(
            *[ResidualBlock(256) for _ in range(num_residual)]
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(256, 128, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(128),
            nn.ReLU(inplace=True),

            nn.ConvTranspose2d(128, 64, 3, stride=2, padding=1, output_padding=1),
            nn.InstanceNorm2d(64),
            nn.ReLU(inplace=True),

            nn.ReflectionPad2d(3),
            nn.Conv2d(64, out_channels, 7),
            nn.Tanh()
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.residual_blocks(x)
        x = self.decoder(x)
        return x

In [21]:
# ==================== Discriminator ====================
class Discriminator(nn.Module):
    def __init__(self, in_channels=3):
        super().__init__()

        def discriminator_block(in_filters, out_filters, normalize=True):
            layers = [nn.Conv2d(in_filters, out_filters, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *discriminator_block(in_channels, 64, normalize=False),
            *discriminator_block(64, 128),
            *discriminator_block(128, 256),
            *discriminator_block(256, 512),
            nn.Conv2d(512, 1, 4, padding=1)
        )

    def forward(self, img):
        return self.model(img)


In [22]:
# ==================== Perceptual Loss ====================
class PerceptualLoss(nn.Module):
    def __init__(self):
        super().__init__()
        # Use simple MSE for perceptual loss approximation
        self.criterion = nn.MSELoss()

    def forward(self, pred, target):
        return self.criterion(pred, target)


In [23]:
# ==================== Training ====================
class DeblurGANTrainer:
    def __init__(self, train_loader, val_loader, device='cuda'):
        self.device = device
        self.train_loader = train_loader
        self.val_loader = val_loader

        # Models
        self.generator = DeblurGenerator().to(device)
        self.discriminator = Discriminator().to(device)

        # Optimizers
        self.g_optimizer = optim.Adam(self.generator.parameters(), lr=1e-4, betas=(0.5, 0.999))
        self.d_optimizer = optim.Adam(self.discriminator.parameters(), lr=1e-4, betas=(0.5, 0.999))

        # Loss functions
        self.adversarial_loss = nn.MSELoss()
        self.perceptual_loss = PerceptualLoss()
        self.content_loss = nn.L1Loss()

        # Loss weights
        self.lambda_adv = 0.001
        self.lambda_content = 100
        self.lambda_perceptual = 10

        # Tracking
        self.history = {'g_loss': [], 'd_loss': [], 'val_psnr': []}

    def train_epoch(self, epoch):
        self.generator.train()
        self.discriminator.train()

        g_losses = []
        d_losses = []

        pbar = tqdm(self.train_loader, desc=f'Epoch {epoch+1}')
        for blur_imgs, sharp_imgs in pbar:
            blur_imgs = blur_imgs.to(self.device)
            sharp_imgs = sharp_imgs.to(self.device)
            batch_size = blur_imgs.size(0)

            # Train Discriminator
            self.d_optimizer.zero_grad()

            fake_imgs = self.generator(blur_imgs).detach()

            # Get discriminator output to determine size dynamically
            d_out_real = self.discriminator(sharp_imgs)
            d_out_fake = self.discriminator(fake_imgs)

            real_valid = torch.ones_like(d_out_real).to(self.device)
            fake_valid = torch.zeros_like(d_out_fake).to(self.device)

            real_loss = self.adversarial_loss(d_out_real, real_valid)
            fake_loss = self.adversarial_loss(d_out_fake, fake_valid)
            d_loss = (real_loss + fake_loss) / 2

            d_loss.backward()
            self.d_optimizer.step()

            # Train Generator
            self.g_optimizer.zero_grad()

            fake_imgs = self.generator(blur_imgs)
            d_out_fake = self.discriminator(fake_imgs)

            real_valid = torch.ones_like(d_out_fake).to(self.device)

            adv_loss = self.adversarial_loss(d_out_fake, real_valid)
            content_loss = self.content_loss(fake_imgs, sharp_imgs)
            perceptual_loss = self.perceptual_loss(fake_imgs, sharp_imgs)

            g_loss = (self.lambda_adv * adv_loss +
                     self.lambda_content * content_loss +
                     self.lambda_perceptual * perceptual_loss)

            g_loss.backward()
            self.g_optimizer.step()

            g_losses.append(g_loss.item())
            d_losses.append(d_loss.item())

            pbar.set_postfix({'G_loss': f'{g_loss.item():.4f}',
                            'D_loss': f'{d_loss.item():.4f}'})

        return np.mean(g_losses), np.mean(d_losses)

    def validate(self):
        self.generator.eval()
        psnrs = []

        with torch.no_grad():
            for blur_imgs, sharp_imgs in self.val_loader:
                blur_imgs = blur_imgs.to(self.device)
                sharp_imgs = sharp_imgs.to(self.device)

                fake_imgs = self.generator(blur_imgs)

                # Calculate PSNR
                mse = torch.mean((fake_imgs - sharp_imgs) ** 2)
                psnr = 20 * torch.log10(1.0 / torch.sqrt(mse))
                psnrs.append(psnr.item())

        return np.mean(psnrs)

    def train(self, num_epochs=50, save_dir='checkpoints'):
        os.makedirs(save_dir, exist_ok=True)
        best_psnr = 0

        for epoch in range(num_epochs):
            g_loss, d_loss = self.train_epoch(epoch)
            val_psnr = self.validate()

            self.history['g_loss'].append(g_loss)
            self.history['d_loss'].append(d_loss)
            self.history['val_psnr'].append(val_psnr)

            print(f'Epoch {epoch+1}/{num_epochs} - '
                  f'G_loss: {g_loss:.4f}, D_loss: {d_loss:.4f}, '
                  f'Val PSNR: {val_psnr:.2f}')

            # Save best model
            if val_psnr > best_psnr:
                best_psnr = val_psnr
                torch.save({
                    'epoch': epoch,
                    'generator_state_dict': self.generator.state_dict(),
                    'discriminator_state_dict': self.discriminator.state_dict(),
                    'g_optimizer_state_dict': self.g_optimizer.state_dict(),
                    'd_optimizer_state_dict': self.d_optimizer.state_dict(),
                    'best_psnr': best_psnr
                }, f'{save_dir}/best_model.pth')

            # Save checkpoint every 10 epochs
            if (epoch + 1) % 10 == 0:
                torch.save({
                    'epoch': epoch,
                    'generator_state_dict': self.generator.state_dict(),
                    'discriminator_state_dict': self.discriminator.state_dict(),
                }, f'{save_dir}/checkpoint_epoch_{epoch+1}.pth')

        self.plot_history(save_dir)

    def plot_history(self, save_dir):
        fig, axes = plt.subplots(1, 3, figsize=(15, 4))

        axes[0].plot(self.history['g_loss'])
        axes[0].set_title('Generator Loss')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Loss')

        axes[1].plot(self.history['d_loss'])
        axes[1].set_title('Discriminator Loss')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Loss')

        axes[2].plot(self.history['val_psnr'])
        axes[2].set_title('Validation PSNR')
        axes[2].set_xlabel('Epoch')
        axes[2].set_ylabel('PSNR (dB)')

        plt.tight_layout()
        plt.savefig(f'{save_dir}/training_history.png')
        plt.close()


In [24]:
# ==================== Main ====================
def main():
    # Mount Google Drive (for Colab)
    try:
        from google.colab import drive
        drive.mount('/content/drive')
        print("Google Drive mounted successfully!")
    except:
        print("Not running in Colab or Drive already mounted")

    # Configuration
    DATASET_PATH = "/content/drive/MyDrive/CV/gopro_deblur"
    BLUR_DIR = f'{DATASET_PATH}/blur/images'
    SHARP_DIR = f'{DATASET_PATH}/sharp/images'
    BATCH_SIZE = 8  # Increased for T4 GPU
    NUM_EPOCHS = 50
    IMG_SIZE = 256

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')

    # Print GPU info if available
    if torch.cuda.is_available():
        print(f'GPU: {torch.cuda.get_device_name(0)}')
        print(f'GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')

    # Transforms
    transform = transforms.Compose([
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])

    # Dataset
    dataset = GoPRODataset(BLUR_DIR, SHARP_DIR, transform=transform)

    # Split dataset (80% train, 20% val)
    train_size = int(0.8 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(
        dataset, [train_size, val_size]
    )

    print(f'Train size: {len(train_dataset)}, Val size: {len(val_dataset)}')

    # DataLoaders
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE,
                            shuffle=True, num_workers=4, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE,
                          shuffle=False, num_workers=4, pin_memory=True)

    # Save checkpoints to Drive
    CHECKPOINT_DIR = '/content/drive/MyDrive/CV/deblurgan_checkpoints'

    # Train
    trainer = DeblurGANTrainer(train_loader, val_loader, device=device)
    trainer.train(num_epochs=NUM_EPOCHS, save_dir=CHECKPOINT_DIR)

    print('Training completed!')
    print(f'Checkpoints saved to: {CHECKPOINT_DIR}')

if __name__ == '__main__':
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully!
Using device: cuda
GPU: Tesla T4
GPU Memory: 15.83 GB
Found 1029 blur images
Found 1030 sharp images
Missing in sharp: 0
Missing in blur: 1
Using 1029 matched image pairs
Train size: 823, Val size: 206


Epoch 1: 100%|██████████| 103/103 [02:27<00:00,  1.43s/it, G_loss=13.0267, D_loss=0.7036]


Epoch 1/50 - G_loss: 14.9919, D_loss: 0.2973, Val PSNR: 15.50


Epoch 2: 100%|██████████| 103/103 [01:35<00:00,  1.08it/s, G_loss=11.9316, D_loss=0.1747]


Epoch 2/50 - G_loss: 10.8934, D_loss: 0.2053, Val PSNR: 17.51


...

Epoch 48/50 - G_loss: 5.1988, D_loss: 0.0055, Val PSNR: 21.40


Epoch 49: 100%|██████████| 103/103 [01:34<00:00,  1.09it/s, G_loss=4.5201, D_loss=0.0017]


Epoch 49/50 - G_loss: 5.2175, D_loss: 0.0029, Val PSNR: 21.36


Epoch 50: 100%|██████████| 103/103 [01:34<00:00,  1.09it/s, G_loss=6.6057, D_loss=0.0190]


Epoch 50/50 - G_loss: 5.1832, D_loss: 0.0039, Val PSNR: 21.47
Training completed!
Checkpoints saved to: /content/drive/MyDrive/CV/deblurgan_checkpoints
